# 1、基础使用

举例：


In [1]:
# 1.导入相关依赖
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
import os
import dotenv
dotenv.load_dotenv()


# 2.定义文档加载器
loader = TextLoader(file_path='./asset/load/09-ai1.txt',encoding="utf-8")

# 3.加载文档
documents = loader.load()

# 4.定义文本切割器
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 5.切割文档
docs = text_splitter.split_documents(documents)

# 6.定义嵌入模型
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large"
)


In [2]:
# 获取向量数据库
db = FAISS.from_documents(documents=docs,embedding=embeddings)

# 基于向量数据库获取检索器
retriever = db.as_retriever()

# 进行数据的检索
docs = retriever.invoke(input = "深度学习是什么？")

print(len(docs))

for doc in docs:
    print(f"------{doc}")

3
------page_content='人工智能综述：发展、应用与未来展望

摘要
人工智能（Artificial Intelligence，AI）作为计算机科学的一个重要分支，近年来取得了突飞猛进的发展。本文综述了人工智能的发展历程、核心技术、应用领域以及未来发展趋势。通过对人工智能的定义、历史背景、主要技术（如机器学习、深度学习、自然语言处理等）的详细介绍，探讨了人工智能在医疗、金融、教育、交通等领域的应用，并分析了人工智能发展过程中面临的挑战与机遇。最后，本文对人工智能的未来发展进行了展望，提出了可能的突破方向。

1. 引言
人工智能是指通过计算机程序模拟人类智能的一门科学。自20世纪50年代诞生以来，人工智能经历了多次起伏，近年来随着计算能力的提升和大数据的普及，人工智能技术取得了显著的进展。人工智能的应用已经渗透到日常生活的方方面面，从智能手机的语音助手到自动驾驶汽车，从医疗诊断到金融分析，人工智能正在改变着人类社会的运行方式。

2. 人工智能的发展历程
2.1 早期发展
人工智能的概念最早可以追溯到20世纪50年代。1956年，达特茅斯会议（Dartmouth Conference）被认为是人工智能研究的正式开端。在随后的几十年里，人工智能研究经历了多次高潮与低谷。早期的研究主要集中在符号逻辑和专家系统上，但由于计算能力的限制和算法的不足，进展缓慢。
2.2 机器学习的兴起
20世纪90年代，随着统计学习方法的引入，机器学习逐渐成为人工智能研究的主流。支持向量机（SVM）、决策树、随机森林等算法在分类和回归任务中取得了良好的效果。这一时期，机器学习开始应用于数据挖掘、模式识别等领域。
2.3 深度学习的突破
2012年，深度学习在图像识别领域取得了突破性进展，标志着人工智能进入了一个新的阶段。深度学习通过多层神经网络模拟人脑的工作方式，能够自动提取特征并进行复杂的模式识别。卷积神经网络（CNN）、循环神经网络（RNN）和长短期记忆网络（LSTM）等深度学习模型在图像处理、自然语言处理、语音识别等领域取得了显著成果。' metadata={'source': './asset/load/09-ai1.txt'}
------page_content='3. 人工智能的核心技术
3.1 机器学习
机器学习是人工智能的核心技术之一，通过算法使计算机从数

# 2、体会不同的检索策略



In [3]:
# 1.导入相关依赖
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

# 2.定义文档
document_1 = Document(
    page_content="经济复苏：美国经济正在从疫情中强劲复苏，失业率降至历史低点。！",
)
document_2 = Document(
    page_content="基础设施：政府将投资1万亿美元用于修复道路、桥梁和宽带网络。",
)
document_3 = Document(
    page_content="气候变化：承诺到2030年将温室气体排放量减少50%。",
)
document_4 = Document(
    page_content=" 医疗保健：降低处方药价格，扩大医疗保险覆盖范围。",
)
document_5 = Document(
    page_content="教育：提供免费的社区大学教育。。",
)
document_6 = Document(
    page_content="科技：增加对半导体产业的投资以减少对外国供应链的依赖。。",
)
document_7 = Document(
    page_content="外交政策：继续支持乌克兰对抗俄罗斯的侵略。",
)
document_8 = Document(
    page_content="枪支管制：呼吁国会通过更严格的枪支管制法律。",
)
document_9 = Document(
    page_content="移民改革：提出全面的移民改革方案。",
)
document_10 = Document(
    page_content="社会正义：承诺解决系统性种族歧视问题。",
)
documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

# 3.创建向量存储
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large"
)

# 4.将文档向量化，添加到向量数据库索引中，得到向量数据库对象
db = FAISS.from_documents(documents, embeddings)

① 默认检索器使用相似性搜索

In [4]:
# 获取检索器
retriever = db.as_retriever(search_kwargs={"k": 3}) #这里设置返回的文档数

docs = retriever.invoke("经济政策")

for i, doc in enumerate(docs):
    print(f"\n结果 {i+1}:\n{doc.page_content}\n")



结果 1:
经济复苏：美国经济正在从疫情中强劲复苏，失业率降至历史低点。！


结果 2:
科技：增加对半导体产业的投资以减少对外国供应链的依赖。。


结果 3:
外交政策：继续支持乌克兰对抗俄罗斯的侵略。



② 分数阈值查询

In [5]:
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.1}
)
docs = retriever.invoke("经济政策")

for doc in docs:
    print(f"📌 内容: {doc.page_content}")

📌 内容: 经济复苏：美国经济正在从疫情中强劲复苏，失业率降至历史低点。！


③ MMR搜索

In [7]:
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={"fetch_k":2}
)

docs = retriever.invoke("经济政策")

print(len(docs))

for  doc  in  docs:
    print(f"📌 内容: {doc.page_content}")

2
📌 内容: 经济复苏：美国经济正在从疫情中强劲复苏，失业率降至历史低点。！
📌 内容: 科技：增加对半导体产业的投资以减少对外国供应链的依赖。。


# 3、结合大模型的使用

举例1：不使用RAG技术

In [8]:
from langchain_openai import ChatOpenAI
import os
import dotenv
dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")

# 创建大模型实例
llm = ChatOpenAI(model="gpt-4o-mini")

# 调用
response = llm.invoke("北京有什么著名的建筑？")
print(response.content)

北京作为中国的首都，拥有众多著名的建筑。以下是一些代表性的建筑：

1. **故宫**：汉族古代宫殿建筑的杰出代表，是世界上保存最完好的古代皇宫之一。

2. **天安门广场和天安门**：是中国的象征之一，广场是世界上最大的城市广场，天安门则是重要的历史和政治象征。

3. **长城**：虽然长城跨越多个省份，但北京境内的八达岭和慕田峪段是最为著名的游览区。

4. **颐和园**：中国古代皇家园林，以其精美的建筑和自然景观而闻名。

5. **鸟巢（国家体育场）**：2008年北京奥运会的主场馆，以其独特的建筑设计而受到广泛关注。

6. **水立方（国家游泳中心）**：同样是2008年奥运会的赛事场馆，以其独特的水泡外观而闻名。

7. **天坛**：明清时期皇帝祭天祈谷的地方，以其独特的建筑和园林设计而享有盛名。

8. **中轴线建筑群**：包括钟楼、鼓楼等，是北京城市规划的核心部分，具有重要的历史和文化价值。

9. **北京首都博物馆**：现代建筑的代表之一，展示了丰富的历史与文化。

10. **中国中央电视台总部大楼**（CCTV headquarters）：以其独特的造型和现代化设计而著称。

这些建筑不仅展示了北京的历史与文化底蕴，也展现了现代建筑的魅力。


举例2：通过FAISS构建一个可搜索的向量索引数据库，并结合RAG技术让LLM去回答问题。

In [9]:
# 1. 导入所有需要的包
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
import os
import dotenv

dotenv.load_dotenv()

# 2. 创建自定义提示词模板
prompt_template = """请使用以下提供的文本内容来回答问题。仅使用提供的文本信息，如果文本中没有相关信息，请回答"抱歉，提供的文本中没有这个信息"。

文本内容：
{context}

问题：{question}

回答：
"
"""

prompt = PromptTemplate.from_template(prompt_template)

# 3. 初始化模型
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

# 4. 加载文档
loader = TextLoader("./asset/load/10-test_doc.txt", encoding='utf-8')
documents = loader.load()

# 5. 分割文档
text_splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
)
texts = text_splitter.split_documents(documents)

#print(f"文档个数:{len(texts)}")

# 6. 创建向量存储
vectorstore = FAISS.from_documents(
    documents=texts,
    embedding=embedding_model
)

# 7.获取检索器
retriever = vectorstore.as_retriever()

docs = retriever.invoke("北京有什么著名的建筑？")

# 8. 创建Runnable链
chain = prompt | llm

# 9. 提问
result = chain.invoke(input={"question":"北京有什么著名的建筑？","context":docs})
print("\n回答:", result.content)


回答: 北京有以下著名的建筑：

1. 故宫 - 明清两代的皇家宫殿，世界上现存规模最大、保存最完整的木质结构古建筑群之一。
2. 天安门 - 北京的标志性建筑之一，天安门广场是世界上最大的城市广场。
3. 颐和园 - 清朝时期的皇家园林，融合了江南园林的设计风格。
4. 天坛 - 明清两代皇帝祭天、祈谷的场所，具有深厚的文化内涵。
5. 长城（八达岭段） - 最著名的北京段，被誉为"世界第八大奇迹"。
6. 国家体育场（鸟巢） - 2008年奥运会主体育场，以独特的钢结构设计闻名。
7. 中央电视台总部大楼 - 现代北京最具争议和识别度的建筑之一。
8. 国家大剧院 - 因其蛋壳造型被称为"巨蛋"，是世界最大的穹顶建筑之一。
9. 北京大兴国际机场 - 超大型国际航空枢纽，被誉为"新世界七大奇迹"之一。
10. 鼓楼和钟楼 - 古代中国的计时中心，展现了古代的计时智慧。
